1. Chargement des données de Bordeaux

In [7]:
# 📦 Imports (si pas déjà faits)
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import pickle
import os


In [8]:
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import mean_squared_error, r2_score

# Charger le bundle complet
models_bundle = joblib.load('../models/model_lille.pkl')

# Charger scalers
scaler_X_appart = joblib.load('../models/scaler_X_appart.pkl')
scaler_y_appart = joblib.load('../models/scaler_y_appart.pkl')
scaler_X_maison = joblib.load('../models/scaler_X_maison.pkl')
scaler_y_maison = joblib.load('../models/scaler_y_maison.pkl')

# Extraire dictionnaires de modèles
models_appart = models_bundle['models_appart']
models_maison = models_bundle['models_maison']

# Charger données Bordeaux
df = pd.read_csv('../data/bordeaux_2022.csv')

# Calcul prix_m2
df['prix_m2'] = df['Valeur fonciere'] / df['Surface reelle bati']

# Filtrer logements 4 pièces
df = df[df['Nombre pieces principales'] == 4].copy()

# Colonnes utiles (exclure "Maison" des features car pas dans scaler)
features = ['Surface reelle bati', 'Nombre de lots', 'Surface terrain']

# Nettoyer NaN
df = df.dropna(subset=features + ['prix_m2', 'Type local'])

# Séparer appartements et maisons
df_appart = df[df['Type local'] == 'Appartement']
df_maison = df[df['Type local'] == 'Maison']

def prepare_X(df_sub, scaler_X):
    X = df_sub[features]
    return scaler_X.transform(X)

def prepare_y(df_sub, scaler_y):
    y_true = df_sub['prix_m2'].values.reshape(-1, 1)
    y_scaled = scaler_y.transform(y_true)
    return y_true, y_scaled

def evaluate(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    return mse, rmse, r2

# Préparation données appartement
X_appart = prepare_X(df_appart, scaler_X_appart)
y_appart_true, _ = prepare_y(df_appart, scaler_y_appart)

# Préparation données maison
X_maison = prepare_X(df_maison, scaler_X_maison)
y_maison_true, _ = prepare_y(df_maison, scaler_y_maison)

# Fonction pour tester tous les modèles et stocker résultats
def test_models(models_dict, X, y_true, scaler_y):
    results = {}
    for name, model in models_dict.items():
        y_pred_scaled = model.predict(X)
        y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1))
        mse, rmse, r2 = evaluate(y_true, y_pred)
        results[name] = {'MSE': mse, 'RMSE': rmse, 'R2': r2}
    return results

# Tester tous les modèles appartements
results_appart = test_models(models_appart, X_appart, y_appart_true, scaler_y_appart)

# Tester tous les modèles maisons
results_maison = test_models(models_maison, X_maison, y_maison_true, scaler_y_maison)

# Afficher résultats
print("Résultats Appartements :")
for model_name, metrics in results_appart.items():
    print(f"{model_name}: MSE={metrics['MSE']:.4f}, RMSE={metrics['RMSE']:.4f}, R2={metrics['R2']:.4f}")

print("\nRésultats Maisons :")
for model_name, metrics in results_maison.items():
    print(f"{model_name}: MSE={metrics['MSE']:.4f}, RMSE={metrics['RMSE']:.4f}, R2={metrics['R2']:.4f}")


Résultats Appartements :
LinearRegression: MSE=562941695.8564, RMSE=23726.3924, R2=-2.1429
DecisionTree: MSE=363498251.1391, RMSE=19065.6301, R2=-1.0294
RandomForest: MSE=210173841.6200, RMSE=14497.3736, R2=-0.1734
DecisionTree_Optimized: MSE=254473687.4880, RMSE=15952.2314, R2=-0.4207
RandomForest_Optimized: MSE=210173841.6200, RMSE=14497.3736, R2=-0.1734
XGBoost: MSE=253394836.2934, RMSE=15918.3805, R2=-0.4147

Résultats Maisons :
LinearRegression: MSE=12410485.9232, RMSE=3522.8520, R2=-1.2391
DecisionTree: MSE=13458818.9784, RMSE=3668.6263, R2=-1.4282
RandomForest: MSE=12093062.5621, RMSE=3477.5081, R2=-1.1818
DecisionTree_Optimized: MSE=12578783.8840, RMSE=3546.6581, R2=-1.2694
RandomForest_Optimized: MSE=12058202.9212, RMSE=3472.4923, R2=-1.1755
XGBoost: MSE=13100248.7718, RMSE=3619.4266, R2=-1.3635


# Résumé des résultats

Les modèles ont été entraînés sur des données immobilières de Lille, puis testés sur des données de Bordeaux.

- Pour les **appartements**, tous les modèles affichent un score R² négatif, ce qui signifie que leurs prédictions sont moins fiables qu’une estimation naïve (comme la moyenne). Le modèle RandomForest est le plus performant, mais reste insuffisant.

- Pour les **maisons**, la situation est similaire : les erreurs sont plus faibles que pour les appartements, mais le R² reste négatif pour tous les modèles, indiquant une mauvaise capacité à généraliser à Bordeaux.

**Conclusion :**  
Le transfert des modèles de Lille vers Bordeaux ne fonctionne pas bien. Cela montre que les marchés immobiliers sont trop différents entre ces deux villes, et que le modèle ne parvient pas à capturer ces différences. Pour obtenir de bons résultats, il faudra entraîner ou ajuster les modèles directement avec des données locales bordelaises.




In [9]:
import pickle

# ✅ Choix manuels des meilleurs modèles selon les résultats
best_model_appart_name = 'RandomForest'  # ou 'RandomForest_Optimised'
best_model_maison_name = 'LinearRegression'  #

# ✅ Récupération des meilleurs modèles dans les dictionnaires
best_model_appart = models_appart[best_model_appart_name]
best_model_maison = models_maison[best_model_maison_name]

# ✅ Récupération des résultats correspondants
best_results_appart = results_appart[best_model_appart_name]
best_results_maison = results_maison[best_model_maison_name]

# ✅ Création du bundle à sauvegarder
bordeaux_bundle = {
    'best_model_appart': best_model_appart,
    'best_model_maison': best_model_maison,
    'scaler_X_appart_lille': scaler_X_appart,
    'scaler_y_appart_lille': scaler_y_appart,
    'scaler_X_maison_lille': scaler_X_maison,
    'scaler_y_maison_lille': scaler_y_maison,
    'features': features,
    'results_appart': best_results_appart,
    'results_maison': best_results_maison,
     'best_model_appart': best_model_appart,
    'best_model_maison': best_model_maison,
    'scaler_X_appart_bx': scaler_X_appart,
    'scaler_y_appart_bx': scaler_y_appart,
    'scaler_X_maison_bx': scaler_X_maison,
    'scaler_y_maison_bx': scaler_y_maison,
}

# ✅ Sauvegarde dans un fichier .pkl
with open('../models/model_bordeaux_best_2.pkl', 'wb') as f:
    pickle.dump(bordeaux_bundle, f)

print("✅ Bundle Bordeaux sauvegardé dans : '../models/model_bordeaux_best_2.pkl'")



✅ Bundle Bordeaux sauvegardé dans : '../models/model_bordeaux_best_2.pkl'


In [10]:
import joblib

bordeaux_bundle = joblib.load('../models/model_bordeaux_best_2.pkl')
print(bordeaux_bundle.keys())  # Affiche toutes les clés disponibles


dict_keys(['best_model_appart', 'best_model_maison', 'scaler_X_appart_lille', 'scaler_y_appart_lille', 'scaler_X_maison_lille', 'scaler_y_maison_lille', 'features', 'results_appart', 'results_maison', 'scaler_X_appart_bx', 'scaler_y_appart_bx', 'scaler_X_maison_bx', 'scaler_y_maison_bx'])
